# Enhancing RAG with Neo4j Knowledge Graph


Import modules and instantiate connections and models

In [21]:
import snowflake.connector
from langchain.vectorstores.neo4j_vector import Neo4jVector
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

import config
import graph


embedding_model = HuggingFaceEmbeddings(
    model_name=config.EMBEDDING_MODEL_CHECKPOINT,
    model_kwargs=dict(trust_remote_code=True, revision=config.EMBEDDING_MODEL_HASH)
)

vector_index = Neo4jVector.from_existing_index(
    embedding_model,
    url=graph.URI,
    username=graph.USERNAME,
    password=graph.PASSWORD,
    index_name='chunkEmbedding',
    keyword_index_name='chunkText',
    search_type='vector'
)

In [22]:
question = 'What does the AfD want to do?'
# filter = {'category': {"$eq": "summary"}}
retrieved_docs = vector_index.similarity_search(question, k=5, filter=None)
for doc in retrieved_docs:
    print(doc)

page_content='Will die AfD in einem hessischen Ausschuss zu einem Rundumschlag gegen die gesamte deutsche Corona-Politik ausholen? Im Streit um die Verfassungsmäßigkeit ihrer Initiative hat sie alle drei Gutachter gegen sich, die der Landtag beauftragt hat.' metadata={'position': 0, 'uid': 'Chunk:IizILhutSDSy', 'category': 'summary', 'section': 0}
page_content='Freude bei der AfD: Sie landet bei der Europawahl vor SPD und Grünen. Auf der Wahlparty heißt es, man wolle das EU-Parlament „eindampfen“. Aber in welcher Fraktion eigentlich?' metadata={'position': 0, 'uid': 'Chunk:Qy0YywgPSX--', 'category': 'summary', 'section': 0}
page_content='Die AfD wird bei der Europawahl zweitstärkste Kraft in Deutschland, in Ostdeutschland liegt sie sogar vorn. Doch ungeachtet dieses Erfolgs steuert sie im Europaparlament in eine ungewisse Zukunft: Die Fraktionslosigkeit droht. Damit gingen mehrere Nachteile einher.' metadata={'position': 0, 'uid': 'Chunk:4F6Y1MUmQ6ax', 'category': 'summary', 'section':

In [23]:
vector_index = Neo4jVector.from_existing_index(
    embedding_model,
    url=graph.URI,
    username=graph.USERNAME,
    password=graph.PASSWORD,
    index_name='chunkEmbedding',
    keyword_index_name='chunkText',
    search_type='hybrid'
)
retrieved_docs = vector_index.similarity_search(question, k=5, filter=None)
for doc in retrieved_docs:
    print(doc)

page_content='Die AfD wolle sich in Brüssel darauf konzentrieren, das EU-Parlament „einzudampfen“, vor allem, was Mitarbeiter angehe. Denn diese würden von deutschen Steuerzahlern finanziert, so Chrupalla. Außerdem sollten Entscheidungen wieder zurück auf die nationale Ebene verlegt werden. Alice Weidel erklärt sich das gute Ergebnis auch mit wachsender Kritik der Bevölkerung an Europa.' metadata={'position': 8, 'uid': 'Chunk:-FyuRf0HT3a3', 'category': 'paragraph', 'section': 2}
page_content='"That\'s what inspired me to want to make change. I want to do something about it." He is best known for starring on Gogglebox alongside his sister Amy, mum Nikki and dad Jonathan and he looks totally grown up compared to his days filming the show.' metadata={'position': 9, 'uid': 'Chunk:6hpXGzxMTW6H', 'category': 'paragraph', 'section': 1}
page_content='19.00 Uhr: CSU will AfD mit AfD-Politik bekämpfen' metadata={'position': 147, 'uid': 'Chunk:pRc5lKPaTHOp', 'category': 'headline', 'section': 44}

In [31]:
retrieval_query=(
    "WITH node as chunk, score "
    "MATCH (chunk)<-[:CONTAINS]-(a)<-[:PUBLISHED]-(s) "
    "WITH chunk, score, a, s "
    "RETURN 'Title: ' + a.title + '\\nText: ' + chunk.text as text, score, "
    "chunk{.position, .section, .category, date: a.publishing_date, url:a.url, source: s.name} as metadata"
)
print(retrieval_query)
vector_index = Neo4jVector.from_existing_index(
    embedding_model,
    url=graph.URI,
    username=graph.USERNAME,
    password=graph.PASSWORD,
    index_name='chunkEmbedding',
    keyword_index_name='chunkText',
    retrieval_query=retrieval_query
)
retrieved_docs = vector_index.similarity_search(question, k=5, filter=None)
for doc in retrieved_docs:
    print(doc)

WITH node as chunk, score MATCH (chunk)<-[:CONTAINS]-(a)<-[:PUBLISHED]-(s) WITH chunk, score, a, s RETURN 'Title: ' + a.title + '\nText: ' + chunk.text as text, score, chunk{.position, .section, .category, date: a.publishing_date, url:a.url, source: s.name} as metadata
page_content='Title: Die AfD jubelt in Berlin über ihren Wahlerfolg – doch wo ist Maximilian Krah?\nText: Die AfD wolle sich in Brüssel darauf konzentrieren, das EU-Parlament „einzudampfen“, vor allem, was Mitarbeiter angehe. Denn diese würden von deutschen Steuerzahlern finanziert, so Chrupalla. Außerdem sollten Entscheidungen wieder zurück auf die nationale Ebene verlegt werden. Alice Weidel erklärt sich das gute Ergebnis auch mit wachsender Kritik der Bevölkerung an Europa.' metadata={'position': 8, 'category': 'paragraph', 'source': 'Berliner Zeitung', 'date': neo4j.time.Date(2024, 6, 9), 'section': 2, 'url': 'https://www.berliner-zeitung.de/politik-gesellschaft/die-afd-jubelt-in-berlin-ueber-ihren-wahlerfolg-doch-wo

In [32]:
import snowflake.connector
from langchain.chains import RetrievalQA

from llm import Cortex

snowflake_connection = snowflake.connector.connect(**config.SNOWFLAKE_CONNECTION_PARAMS)
model = Cortex(connection=snowflake_connection, model=config.CHAT_MODEL)

vector_qa = RetrievalQA.from_chain_type(
    llm=model,
    chain_type='stuff',
    retriever=vector_index.as_retriever()
)

In [33]:
vector_qa.run(question)

' The AfD wants to focus on draining the EU Parliament, especially concerning its employees, as they are financed by German taxpayers, according to Chrupalla. They also aim to move decision-making back to the national level. Additionally, the AfD advocates for a national currency and wishes to end the use of the Euro.'

## Generate a Cypher query

In [12]:
from langchain.chains import GraphCypherQAChain

db = graph.NewsGraphClient()

cypher_chain = GraphCypherQAChain.from_llm(
    cypher_llm = model,
    qa_llm = model,
    graph=db.graph,
    verbose=True
)


In [13]:
cypher_chain.run(question)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
 MATCH (p:Person {name: "Ursula von der Leyen"})-[:AUTHORED]->(a:Article) RETURN a.title, a.publishing_date, a.url LIMIT 10
Full Context:
[]

> Finished chain.


" I don't know the answer based on the provided information."